# Named-E

GNI 코퍼스로 부터 약어와 특수용어들을 가져온다.

In [81]:
import nltk
from nltk.corpus import *
from nltk import *
from nltk import RegexpTokenizer

# using sentence tokenized corpus
corpus_root = 'C:/Users/Maeg/nltk_data/Genomics-Informatics-Corpus-master/Genomics-Informatics-Corpus-master/GNI Corpus 1.0/sentence_tokenized'
GNIcorpus = PlaintextCorpusReader(corpus_root,'.*\.txt',encoding='utf-8')
raws = GNIcorpus.raw()

<b>1. Tokenization: 약어들과 일반적인 단어, 수치들이 나오도록 tokenziation한다.</b> <br>
pattern : GNI corpus을 tokenization 하기 위해서 설정한 정규표현식<br>
gi_words : tokenization된 G&I corpus

In [82]:
pattern = r''' (?x)
          \$?\d+(?:\.\d+)?%?      # digits, percentags, and $
        | [a-z]+[A-Z]{2,}            # small letters + large letters abb
        | [A-Za-z]+[0-9]*         # letters + digit abb
        | [A-Z]{2,}[a-z]+         # large letters + small letters abb
        | [A-Z]{2,}               # Large scale abb
        | (?=[A-Z]\.)+            # abb with dots. e.g.  U.S.A
        | [A-Za-z]+(?:-\w+)*            # general token with hyphens
        | (?![\]\[.,;\"\'?\(\):-_`&]) # do not capture separator
'''
tokenizer = RegexpTokenizer(pattern)
gi_words = tokenizer.tokenize(raws)
gi_words = [w for w in gi_words if (w != '')]

<b>2. Preprocessing: 유전자 sequence를 나타내는 문자열 제거<br></b>
re_genecode : 유전자 sequence를 표현하는 정규표현식

In [83]:
re_genecode = r'[AGCT]{3,}[d]?'
gi_word = [w for w in gi_words if (not re.match(re_genecode, w))]

# 태깅을 위한 각 클래스별 약어 Dictionary 생성
 Biology: https://www.abbreviations.com/acronyms/BIOLOGY
  <br> Statistics: https://www.abbreviations.com/acronyms/STATISTICS
  <br> Gene: https://www.genenames.org/
  <br> protein: https://www.cellsignal.com/contents/resources/protein-and-pathway-abbreviations/pathway-abbreviations
  <br><br> 위 링크에서 얻은 소스들로 Biology와 Statistics, gene, protein 4가지 약어 Dictionary를 생성한다.
  <br>

In [84]:
# 소스들이 있는 path
f_root = 'C:/Users/Maeg/AI/Abbreviation'
f1_root = 'C:/Users/Maeg/AI/genename'

bio_abb_corpus = PlaintextCorpusReader(f_root,'Biology Abbreviations.txt')
stat_abb_corpus = PlaintextCorpusReader(f_root,'statistics Abbreviations.txt')
genename_corpus = PlaintextCorpusReader(f1_root,'genenames.txt')
pro_abb_corpus = PlaintextCorpusReader(f1_root,'proteinname.txt')

# 각 소스로부터 약어를 뽑아온다.
bio_abb = ('\r\n'.join(bio_abb_corpus.raw().split('\t'))).split('\r\n')
stat_abb = ('\r\n'.join(stat_abb_corpus.raw().split('\t'))).split('\r\n')
genenames = genename_corpus.raw().split('\n')
pro_abb_temp = pro_abb_corpus.raw().split('\r\n')
pro_abb = []
for i in range(len(pro_abb_temp)):
    s = re.findall(r'([A-Za-z-0-9]+)\t', pro_abb_temp[i])
    pro_abb.append(s[0])

In [85]:
# 해당 약어가 포함된 클래스를 리턴, 어떤 클래스에도 속하지 않을경우 0을 리턴한다.
def Check_class(str):
    count = 0
    classes = []
    
    if str in bio_abb:
        count += 1
        classes.append('biology')
    if str in stat_abb:
        count += 1
        classes.append('statistical')
    if str in pro_abb:
        count += 1
        classes.append('protein')
    if str in genenames:
        count += 1
        classes.append('gene')
    
    if count >= 1:
        return classes
    else:
        return 0

# 문맥을 고려해 클래스 지정함수 만들기
https://en.wikipedia.org/wiki/Biology <br>
https://en.wikipedia.org/wiki/Gene <br>
https://en.wikipedia.org/wiki/Glossary_of_biology<br>
https://en.wikipedia.org/wiki/Glossary_of_probability_and_statistics<br>
https://en.wikipedia.org/wiki/Protein<br>
https://en.wikipedia.org/wiki/Statistics<br>

약어 주변에 어떤 단어들이 오는지 확인하고 4가지 클래스와 연관성을 도출하기 위해서 Word2Vec모델을 만든다.<br>
Word2Vec모델을 만드는데에 위 링크의 글들을 활용한다.

In [86]:
from gensim.models import Word2Vec

In [87]:
path= 'C:/Users/Maeg/AI/data'
corpus = PlaintextCorpusReader(path,".*\.txt")

#학습 문장 생성
pre_sentences = corpus.sents()

#lower preprocessing
sentences = []
for sent in pre_sentences :
    s = []
    for word in sent:
        s.append(word.lower())
    sentences.append(s)

model = Word2Vec(sentences, size=300, window=3, min_count=1, workers=1)
word_vectors = model.wv
vocabs = list(word_vectors.vocab.keys())
classes = ['biology', 'gene', 'protein', 'statistical']
sentences

[['glossary',
  'of',
  'biology',
  '-',
  'wikipedia',
  'this',
  'glossary',
  'of',
  'biology',
  'terms',
  'is',
  'a',
  'list',
  'of',
  'definitions',
  'of',
  'fundamental',
  'terms',
  'and',
  'concepts',
  'used',
  'in',
  'biology',
  ',',
  'the',
  'study',
  'of',
  'life',
  'and',
  'of',
  'living',
  'organisms',
  '.'],
 ['it',
  'is',
  'intended',
  'as',
  'introductory',
  'material',
  'for',
  'novices',
  ';',
  'for',
  'more',
  'specific',
  'and',
  'technical',
  'definitions',
  'from',
  'sub',
  '-',
  'disciplines',
  'and',
  'related',
  'fields',
  ',',
  'see',
  'glossary',
  'of',
  'genetics',
  ',',
  'glossary',
  'of',
  'ecology',
  ',',
  'glossary',
  'of',
  'speciation',
  ',',
  'glossary',
  'of',
  'botany',
  ',',
  'and',
  'glossary',
  'of',
  'scientific',
  'naming',
  '.'],
 ['contents',
  ':',
  'the',
  'shedding',
  'of',
  'flowers',
  ',',
  'leaves',
  ',',
  'and',
  '/',
  'or',
  'fruit',
  'following',
  'fo

In [88]:

# return tag of abbreviation. with around wording.
def TOA_GNI(str, classes):
    if str in gi_word :
        # 해당 단어가 GNI 코퍼스 상에서 위치하는 곳
        
        idx = gi_word.index(str)
        concordance = gi_word[idx-3:idx+4] # 주변단어들 : 앞뒤로 3개의 단어들.
        
        # 이전 Check_Class에서의 속한 클래스가 있었다면 해당 클래스에 대해서만 연관성 분석
        if classes == 0 :
            cla = ['biology', 'gene', 'protein', 'statistical']
        else :
            cla = classes
        
        max_class = ''
        max_sim = -1
        
        # 최고 연관도(주변단어들과 class의 연관도의 평균)를 구한다.
        for c in cla :
            count = 0;
            sim = 0;
            for w in concordance :
                # 주변단어일 경우, 모델에 해당 단어가 있는지 확인하고 연관도를 출력한다.
                #print(w)
                if w != str :
                    # print('\n',w)
                    if w in vocabs :
                        sim += word_vectors.similarity(c, w.lower())
                        count += 1
            if count != 0 :
                sim /= count # 연관도의 평균
            if max_sim < sim :
                max_sim = sim
                max_class = c
        #print()
        
        return (max_class,max_sim)
    
    #str을 참조할수 없는 경우 태깅 불가
    else :
        return 0

# 약어 태깅하기

In [89]:
abb_pattern = r''' (?x)
         [a-z]+[A-Z]{2,}            # small letters + large letters abb
        | [A-Za-z]+[0-9]+         # letters + digit abb
        | [A-Z]{2,}[a-z]+         # large letters + small letters abb
        | [A-Z]{2,}               # Large scale abb
'''
abb_selector = RegexpTokenizer(abb_pattern)
abb_list = abb_selector.tokenize(raws)
abb_list = [w for w in abb_list if (w != '')]

In [90]:
abb_list= list(set(abb_list))

In [91]:
for i in range(len(abb_list)):
    # 약어를 태깅한다.
    str= abb_list[i]
    cla = Check_class(str)
    
    if cla != 0 :
        if len(cla) > 1:
            print(abb_list[i], TOA_GNI(str, cla),"from more than 2 class")
        else:
            print(abb_list[i], Check_class(Result[i])) 
    else:
        print(abb_list[i], TOA_GNI(str, cla))

rs35408040 ('biology', 0)
Rv2131 ('statistical', 0.9998664259910583)
TAP ('protein', 0.9986611604690552)
EGFR ('biology', 0.9999522566795349) from more than 2 class
DNase ('protein', 0.9993686676025391)
TBE ('statistical', 0.9992097616195679)
SST ('statistical', 0.9985781908035278) from more than 2 class
LCK ('protein', 0.9999417960643768) from more than 2 class
PARP 0
EDNQ ('biology', 0)
heWAS 0
MIx 0
ADInstruments ('statistical', 0.9974109927813212)
CBOW ('protein', 0.9999791383743286)
R50 ('protein', 0.9998643398284912)
PKC 0
P1 0
A5 ('statistical', 0.9993819743394852)
SNPstream ('gene', 0.9908275902271271)
QIAseq ('protein', 0.9995630780855814)
TRP415 ('protein', 0.9999806880950928)
gff3 ('statistical', 0.9933895866076151)
CERs ('protein', 0.9885353296995163)
SAH ('biology', 0)
ciELO 0
DNNClassifier ('statistical', 0.5622685998678207)
i2 ('protein', 0.9999140799045563)
j2 ('biology', 0.9918276220560074)
SY5 ('biology', 0)
SUSD2 0
MODREFINER ('statistical', 0.9978245198726654)
rs127

AMR ('protein', 0.9982264836629232)
ADB ('protein', 0.9998213291168213)
me3 ('biology', 0.9979875087738037)
DFT ('protein', 0.9998232026894888)
PRK ('protein', 0.9994042217731476)
PRC2 ('biology', 0.8954190731048584)
rs2553614 ('biology', 0)
let7 ('statistical', 0.9932810465494791)
CRC ('protein', 0.9992407858371735)
NNSR ('gene', 0.9955798387527466)
CpG2 ('biology', 0.9916562636693319)
CGTG 0
PMP22 0
Cos7 ('protein', 0.9990125447511673)
Ile149 ('protein', 0.9999111493428549)
TAS2 ('biology', 0)
NE ('protein', 0.9907604157924652)
TEN ('gene', 0.9899293780326843)
GCA 0
chi1 ('biology', 0.9853466987609864)
rs263 ('protein', 0.9999449551105499)
CDT ('gene', 0.9801153838634491)
rs343 ('protein', 0.9999532103538513)
CDK5 0
RNAfold ('protein', 0.9971748948097229)
uPA ('biology', 0.9768176525831223)
AFP 0
INBIOMEDvision ('protein', 0.9999806880950928)
rs2334499 ('statistical', 0.9977579116821289)
SLR ('statistical', 0.9993695616722107)
UBC 0
B5 ('protein', 0.9877709746360779)
rs3856726 ('prot

TOMTOM ('protein', 0.9963628351688385)
ACC 0
RE ('gene', 0.9981665313243866)
OG ('protein', 0.9994665533304214)
SP ('biology', 0)
CGM ('protein', 0.9999765753746033)
GMAP ('gene', 0.9884885549545288)
HO ('statistical', 0.9995726346969604)
PDH ('statistical', 0.9998426735401154)
mrFAST ('protein', 0.997119128704071)
MIR548 ('statistical', 0.9985074251890182)
KBr 0
CESA ('protein', 0.9999275207519531)
APeg3 ('protein', 0.9999599456787109)
roSA 0
KANSL2 0
TCHL ('protein', 0.9979006846745809)
rs11150094 ('protein', 0.9999806880950928)
LEF1 0
TPH1 0
TYP ('protein', 0.9990772306919098)
HGSNAT 0
H558 ('biology', 0)
MAT ('protein', 0.9993981271982193)
Cxcl10 ('biology', 0)
LZTR1 0
DCFDA ('gene', 0.999563992023468)
ISP ('protein', 0.9988625645637512)
CD80 0
SMADs ('statistical', 0.9968697786331177)
APBB2 0
GOIs ('biology', 0.9955139756202698)
rs2287780 ('protein', 0.9999806880950928)
PID ('protein', 0.9999806880950928)
phyre2 ('protein', 0.9998892247676849)
RBFOX1 0
BCOR 0
ICTV ('protein', 0.99

PRODRG ('statistical', 0.9987339526414871)
NCC ('protein', 0.9999716877937317)
v8 ('protein', 0.9817325472831726)
OR5 ('biology', 0.9991885721683502)
euTAG ('biology', 0)
DEIs ('protein', 0.9999761581420898)
CCDC137 0
E60 ('protein', 0.9999806880950928)
CREST ('protein', 0.9997480511665344)
PDEs ('protein', 0.9750966131687164)
rs12826786 ('gene', 0.9907355308532715)
MEF 0
ENCODDE ('protein', 0.9896436035633087)
PDBsum ('protein', 0.9998985528945923)
ICD10 ('protein', 0.9996723234653473)
rs55824312 ('protein', 0.9992664456367493)
C6 0
YASARA ('statistical', 0.9957414666811625)
Pkd2 ('gene', 0.9568054527044296)
EDCCs ('protein', 0.9995908339818319)
CAF ('protein', 0.9879342317581177)
SILVA ('protein', 0.9983644336462021)
cuGWAM ('gene', 0.9812209606170654)
ADUL01000064 ('biology', 0.9969098567962646)
B6026918 ('biology', 0)
CHEMBL313493 ('protein', 0.9919567306836446)
rs61769193 ('protein', 0.9999390244483948)
miRC ('protein', 0.9995479186375936)
VECs ('protein', 0.9986245781183243)
rs76

BACs ('protein', 0.9976939658323923)
aTOR 0
SRR8316494 ('biology', 0.9982394774754842)
GWASs ('biology', 0.9870081901550293)
EML4 0
GUI ('biology', 0.990551084280014)
CN ('biology', 0.9356886148452759)
IRF ('gene', 0.9909818887710571)
A4 ('biology', 0.9938552975654602)
SCANPROSITE ('statistical', 0.9985796570777893)
rs80937718 ('protein', 0.9999784628550211)
rs13153032 ('biology', 0)
RAW ('protein', 0.9999026656150818)
IVAG ('biology', 0)
SCHALE ('protein', 0.9960898637771607)
FAIREs ('biology', 0.9975629150867462)
ERS447187 ('protein', 0.9999806880950928)
rs6756959 ('protein', 0.9998834133148193)
tRNAH ('protein', 0.9963626563549042)
ERRATUM ('protein', 0.9999730587005615)
FDRs ('statistical', 0.9982558965682984)
DSM 0
HTR2 ('protein', 0.9997306317090988)
BC11 ('statistical', 0.9766580015420914)
Rs140155614 ('protein', 0.9999599059422811)
deltaBP ('statistical', 0.9405084729194642)
BRCA 0
CDK4 0
E1025 ('protein', 0.9999806880950928)
NMP1 ('statistical', 0.9232668727636337)
ESEs ('stat

...

CVs ('protein', 0.9998123049736023)
GENECODE ('protein', 0.9999179691076279)
BPGWAS ('protein', 0.999211311340332)
rs10228519 ('protein', 0.9999206066131592)
Cas9 ('biology', 0.9995149771372477)
K20 ('protein', 0.9999199509620667)
RCpc ('protein', 0.9999554753303528)
PASS ('protein', 0.9999751448631287)
mDA ('biology', 0.9611385265986124)
ISGs ('protein', 0.9898450970649719)
GWAS ('protein', 0.9931360185146332)
RNP ('statistical', 0.9979910254478455)
HP1 ('protein', 0.9999606370925903)
PDE7 ('protein', 0.9999101608991623)
HMTs ('statistical', 0.9942143360773722)
rs10274486 ('protein', 0.9999168117841085)
CNVtools ('biology', 0.9976915915807089)
RIAs ('gene', 0.9963167905807495)
ennCNV 0
AdoRA2 ('biology', 0)
BFM95 ('statistical', 0.999165877699852)
RNF213 0
xWK 0
SP2 0
COMP 0
CER ('protein', 0.9975714484850565)
NHLBI ('statistical', 0.9889805614948273)
IRF5 0
GP ('protein', 0.9488081137339274)
MAPK ('protein', 0.9928456544876099) from more than 2 class
CCL3 0
rs6715049 ('protein', 0.99

AFLP ('statistical', 0.9929205377896627)
TSD ('protein', 0.9991845786571503)
CpG1437 ('protein', 0.9936893085638682)
HAR1 ('protein', 0.9999253749847412)
IVW ('gene', 0.9575537542502085)
PTC 0
ZNF521 0
Ybx1 ('protein', 0.9998595714569092)
BREs ('statistical', 0.998524621129036)
ITG ('protein', 0.9991373419761658)
IMG ('statistical', 0.9796747416257858)
HL ('statistical', 0.9992300420999527)
ADPKD ('protein', 0.9997796217600504)
PLP1 0
Arg261 ('statistical', 0.9925357580184937)
DZr ('gene', 0.9997618496417999)
BRAFV600 ('protein', 0.996295964717865)
rs41281692 ('protein', 0.9989768713712692)
SDT ('biology', 0.985624098777771)
MLWriting ('statistical', 0.994598925113678)
APHRODITE ('protein', 0.9999206066131592)
PRK15251 ('biology', 0)
IG ('protein', 0.9999211231867472)
PPP ('protein', 0.9953991651535035)
M9 ('biology', 0)
SEPP ('protein', 0.998577606678009)
IIIA ('protein', 0.9892672101656595)
HITS ('statistical', 0.9897105842828751)
rs1393350 ('protein', 0.9999806880950928)
N1 ('protei

BTZ ('protein', 0.9999717473983765)
NPPB 0
RTase ('protein', 0.9998752772808075)
OD260 ('protein', 0.9999716281890869)
Erk1 ('statistical', 0.7919318974018097)
Hiseq2500 ('protein', 0.9999769330024719)
HBGRDs ('protein', 0.998318076133728)
HS408 ('protein', 0.9986562281847)
RA1 ('gene', 0.9954486588637034)
AZ ('biology', 0)
CEBPA 0
HISAT2 ('statistical', 0.9921323180198669)
KRAS 0
PMC ('statistical', 0.9857072085142136)
Pde4 ('biology', 0.920796126127243)
enoSIS 0
TACC3 0
LIZ ('protein', 0.9997752457857132)
MD ('statistical', 0.8689687013626098)
DNAme ('statistical', 0.9998691082000732)
HBe 0
J48 ('biology', 0.8836796134710312)
ZINC03779067 ('gene', 0.9709855914115906)
Wnt2 ('statistical', 0.997013521194458)
Cdc48 ('protein', 0.9999785423278809)
ECNP ('protein', 0.9992243766784668)
GIBCO ('protein', 0.9999771118164062)
O25993 ('protein', 0.9998196760813395)
EULAR ('biology', 0.9991447627544403)
HGDH ('protein', 0.9999806880950928)
GSTs ('statistical', 0.9994161576032639)
GF ('statistic

STAG1 0
ESS 0
KIS ('gene', 0.9983176589012146)
SSPACE ('protein', 0.9775457183519999)
MIR ('statistical', 0.9972390234470367)
rs519113 ('biology', 0)
GPR125 ('protein', 0.9997069001197815)
PRLB ('statistical', 0.999930739402771)
RTE ('protein', 0.9999806880950928)
BI 0
PA ('statistical', 0.999963641166687)
HOX11 ('protein', 0.9999435901641845)
TM3 ('statistical', 0.9839691400527955)
Asn122 ('protein', 0.9999813735485077)
WJSMethodology ('statistical', 0.994598925113678)
CENPF 0
harmacoNER 0
TCGTGACT 0
NEB ('gene', 0.9999014139175415) from more than 2 class
ESSs ('biology', 0.9895806312561035)
IL16 0
CD40 0
WE ('protein', 0.9832274168729782)
L0 ('gene', 0.9910631775856018)
rs6679386 ('gene', 0.9971054792404175)
Elf3 ('protein', 0.9990863800048828)
A1298 ('protein', 0.9999806880950928)
ver8 ('statistical', 0.999963641166687)
AAAAACAACAAGATAA 0
CD36 0
REACTOME ('protein', 0.9985125213861465)
ES ('biology', 0)
OAuth ('statistical', 0.9994672298431396)
CRFs ('gene', 0.5383807222048441)
AI (

ZF ('biology', 0.9978349804878235)
CHIA 0
YSA ('protein', 0.9999449551105499)
CAAT 0
rs33997547 ('protein', 0.9999181926250458)
Cdc42 ('protein', 0.9998984813690186)
VarScan2 ('protein', 0.9999706745147705)
aMM ('protein', 0.9944340586662292)
p25 ('protein', 0.9995392362276713)
CAIcal ('protein', 0.9998733997344971)
IF 0
SWI ('gene', 0.7096202559769154)
DNTAG ('statistical', 0.9984461188316345)
UCP2 0
TCR ('protein', 0.9168461263179779) from more than 2 class
R4 ('biology', 0)
chr20 ('protein', 0.9998980760574341)
DMTF1 0
Bsp1286 ('protein', 0.9999788999557495)
Pin1 ('statistical', 0.8862154662609101)
WHR ('statistical', 0.9994142452875773)
CDC20 0
DUFs ('statistical', 0.9997850855191549)
MIT ('protein', 0.9999784827232361)
CNAT ('protein', 0.9938160181045532)
URIs ('protein', 0.9999310771624247)
LSU ('protein', 0.991961787144343)
TRPMI ('protein', 0.9997009932994843)
ND2 ('protein', 0.9998578429222107)
FABP4 0
DEscription ('protein', 0.9999687969684601)
BB 0
Leu155 ('protein', 0.99901

[('by', 0.999976396560669),
 ('and', 0.9999762773513794),
 ('from', 0.9999752044677734),
 ('with', 0.9999749660491943),
 ('that', 0.9999744892120361),
 ('to', 0.9999744892120361),
 ('on', 0.999974250793457),
 ('in', 0.9999731779098511),
 ('or', 0.9999728202819824),
 ('-', 0.9999728202819824)]